In [1]:
from openai import OpenAI

client = OpenAI(
    # 这里换成你的密钥
    api_key="fk204054-M7aFv2t5pdWOSBMTURsUClih3sPHb8qQ|ck872-e5b3a2d",
    # 这里将官方的接口访问地址，替换成api2d的入口地址
    base_url="https://openai.api2d.net/v1"
)

def get_completion(prompt, stream=False, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages = messages,
        model = model,
        stream = stream
    )
    # 如果是连续输出模式
    if stream==True:
        # 返回一个生成器
        return response
    else:
        # 一次性输出模式
        return response.choices[0].message.content

## 推断主题

大型语言模型的一个很酷的应用是推断主题。给定一段长文本，这段文本是关于什么的？有什么话题？

In [10]:
# 中文
story_zh = """
在政府最近进行的一项调查中，要求公共部门的员工对他们所在部门的满意度进行评分。
调查结果显示，NASA 是最受欢迎的部门，满意度为 95％。

一位 NASA 员工 John Smith 对这一发现发表了评论，他表示：
“我对 NASA 排名第一并不感到惊讶。这是一个与了不起的人们和令人难以置信的机会共事的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，主管 Tom Johnson 表示：
“我们很高兴听到我们的员工对 NASA 的工作感到满意。
我们拥有一支才华横溢、忠诚敬业的团队，他们为实现我们的目标不懈努力，看到他们的辛勤工作得到回报是太棒了。”

调查还显示，社会保障管理局的满意度最低，只有 45％的员工表示他们对工作满意。
政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

## 推断5个主题

上面是一篇虚构的关于政府工作人员对他们工作机构感受的报纸文章。我们可以让它确定五个正在讨论的主题，用一两个字描述每个主题，并将输出格式化为逗号分隔的列表。

In [11]:
# 中文
prompt = f"""
确定以下给定文本中讨论的五个主题。

每个主题用1-2个单词概括。

输出时用逗号分割每个主题。

给定文本: '''{story_zh}'''
"""
response = get_completion(prompt)
print(response)

调查, 满意度, NASA, 员工, 政府


## 为特定主题制作新闻提醒

假设我们有一个新闻网站或类似的东西，这是我们感兴趣的主题：NASA、地方政府、工程、员工满意度、联邦政府等。假设我们想弄清楚，针对一篇新闻文章，其中涵盖了哪些主题。可以使用这样的prompt：确定以下主题列表中的每个项目是否是以下文本中的主题。以 0 或 1 的形式给出答案列表。

In [12]:
# 中文
prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，

以JSON的形式给出答案，每个主题用 0 或 1。

主题列表：美国航空航天局、工资、工程、员工满意度、联邦政府

给定文本: '''{story_zh}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "美国航空航天局": 1,
  "工资": 0,
  "工程": 0,
  "员工满意度": 1,
  "联邦政府": 1
}
```


In [15]:
start = response.find('{')
end = response.rfind('}') + 1
json_str = response[start:end]
json_str

'{\n  "美国航空航天局": 1,\n  "工资": 0,\n  "工程": 0,\n  "员工满意度": 1,\n  "联邦政府": 1\n}'

In [16]:
import json
json_res = json.loads(json_str)
json_res

{'美国航空航天局': 1, '工资': 0, '工程': 0, '员工满意度': 1, '联邦政府': 1}

In [17]:
for key in json_res.keys():
    if json_res[key] == 1:
        print(f"提醒: 关于{key}的新消息")

提醒: 关于美国航空航天局的新消息
提醒: 关于员工满意度的新消息
提醒: 关于联邦政府的新消息
